In [1]:
import requests
import pandas as pd

In [2]:
# Configurações da API
URL_ARTILHEIROS = "https://v3.football.api-sports.io/players/topscorers"
API_KEY = "API_KEY"

In [3]:
# Parâmetros para a requisição
PARAMS = {
    "season": "2022",  # Temporada de interesse
    "league": "71"     # Liga de interesse (ID da liga, ex.: Brasileirao)
}

# Cabeçalhos para autenticação
HEADERS = {
    "x-rapidapi-host": "v3.football.api-sports.io",
    "x-rapidapi-key": API_KEY
}

In [4]:
# Requisição à API
response = requests.get(URL_ARTILHEIROS, headers=HEADERS, params=PARAMS)

In [5]:
# Verificação do status da resposta
if response.status_code == 200:
    # Parseando o JSON retornado
    data = response.json()
    
    # Verificando o campo "response"
    df_artilheiros = data.get("response", [])
    
    if df_artilheiros:
        print(f"Artilheiros encontrados: {len(df_artilheiros)}\n")
        
        # Criando uma lista para armazenar os dados
        players_data = []
        
        # Extraindo informações de cada jogador
        for player in df_artilheiros: 
            player_info = player.get("player", {})
            statistics = player.get("statistics", [{}])[0]
            team = statistics.get("team", {})
            goals = statistics.get("goals", {})
            
            # Adicionando os dados a uma lista como dicionário
            players_data.append({
                "Nome": player_info.get("name", "N/A"),
                "Time": team.get("name", "N/A"),
                "Gols": goals.get("total", 0),
                "Idade": player_info.get("age", "N/A"),
                "Nacionalidade": player_info.get("nationality", "N/A"),
                "Partidas": statistics.get("games", {}).get("appearences", 0),
                "Minutos Jogados": statistics.get("games", {}).get("minutes", 0),
                "Posição": player_info.get("position", "N/A"),
            })
        
        # Criando o DataFrame
        df_artilheiros = pd.DataFrame(players_data)
        
        # Exibindo as primeiras linhas do DataFrame
        print(df_artilheiros.head(31))
    else:
        print("Nenhum artilheiro encontrado.")
else:
    print(f"Erro na requisição: {response.status_code}")
    print("Detalhes:", response.text)

Nenhum artilheiro encontrado.


In [6]:
# Resetando o índice para começar com 1
df_artilheiros.reset_index(drop=True, inplace=True)

# Ajustando o índice para começar com 1
df_artilheiros.index += 1


df_artilheiros

AttributeError: 'list' object has no attribute 'reset_index'

In [ ]:
# Adicionando a coluna "Tempo por Gol" ao DataFrame
df_artilheiros['Tempo por Gol'] = df_artilheiros.apply(
    lambda row: row['Minutos Jogados'] / row['Gols'] if row['Gols'] > 0 else None, axis=1
)

# Exibindo o DataFrame com a nova coluna
df_artilheiros

In [ ]:
# Exportando o DataFrame para um arquivo Excel
# df_artilheiros.to_excel('artilheiros_brasileirao_2022.xlsx', index=False, engine='openpyxl')

# Mensagem de confirmação
print("DataFrame exportado com sucesso para 'artilheiros.xlsx'")